### Reconstruyo la BD donde voy a aplicar la matriz de similitud por falta de memoria en el render

In [16]:
import pandas as pd

# Leo la BD 
f_Games = 'data//Steam-Games(clean).csv'
df_Games = pd.read_csv(f_Games)

print("Games")
total = len(df_Games)
print("tamaño del df original",total)
display(df_Games.head(2))

# Me quedo con las columnas necesarias
df = df_Games[['id','app_name',	'genres']].copy()
# Separo los generos en dif registros
df['genres'] = df['genres'].str.split(',')
df = df.explode('genres')

total = len(df_Games)
print("tamaño del nuevo df",total)
df.head(2)

Games
tamaño del df original 29781


,id,app_name,title,publisher,genres,url,release_date,tags,reviews_url,specs,price,early_access,developer,year
0,761140.0,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,Kotoshiro,2018
1,643980.0,Ironbound,Ironbound,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,Secret Level SRL,2018


tamaño del nuevo df 29781


,id,app_name,genres
0,761140.0,Lost Summoner Kitty,['Action'
0,761140.0,Lost Summoner Kitty,'Casual'


Limpio el campo de genre de espacios y caracteres diferentes a letras y convierto a minusculas

In [31]:

def clean(cadena):
    # Elimina espacios y caracteres que no sean letras y luego convierte a minúsculas
    cadena_limpia = ''.join(caracter.lower() for caracter in cadena if caracter.isalpha())
    return cadena_limpia

df = df[df['genres'].apply(lambda x: isinstance(x, str))]

# limpia la cadena con la funcion clean a la columna 'genres'
df['genres'] = df['genres'].apply(clean)
#Convierte a mayusculas
df['genres'] = df['genres'].str.upper()
df['app_name'] = df['app_name'].str.upper()

#-------------------------------
#Cantidad de valores enteros
num_nan_values_before = df['id'].isna().sum()

#convierto a enteros 'id' y los que no en NaN
df['id'] = pd.to_numeric(df['id'], errors='coerce', downcast='integer')
# Elimino los valores NaN
df = df.dropna(subset=['id'])

#Cantidad de valores enteros
num_nan_values_after = num_nan_values_before - df['id'].isna().sum()
print(f"Se eliminaron {num_nan_values_after} valores NaN de la columna 'id'.")



total = len(df)
print("Numero de registros: ",total)
df.head()

Se eliminaron 0 valores NaN de la columna 'id'.
Numero de registros:  70701


,id,app_name,genres
0,761140,LOST SUMMONER KITTY,ACTION
0,761140,LOST SUMMONER KITTY,CASUAL
0,761140,LOST SUMMONER KITTY,INDIE
0,761140,LOST SUMMONER KITTY,SIMULATION
0,761140,LOST SUMMONER KITTY,STRATEGY


In [33]:
# Guardar el DataFrame en un archivo CSV
total = len(df)
print("Total de registros: ",total)

# Save the DataFrame to a Parquet file
file_path = 'dataRender//SistemaRecomendacion.parquet'
df.to_parquet(file_path, engine='pyarrow')

df.head(3)

Total de registros:  70701


,id,app_name,genres
0,761140,LOST SUMMONER KITTY,ACTION
0,761140,LOST SUMMONER KITTY,CASUAL
0,761140,LOST SUMMONER KITTY,INDIE


### Construye la matriz de similitud
Caracteristicas analizadas:
-   genres



In [42]:
from sklearn.metrics.pairwise import cosine_similarity

# Elegir las características para el cálculo de similitud
features = df.copy()

# Pivotea los datos para obtener una matriz de características por juego
pivot_data = features.pivot_table(index="id", columns=["genres"], aggfunc=lambda x: 1, fill_value=0)

# Calcula la matriz de similitud del coseno
similarities = cosine_similarity(pivot_data)

# Convierte la matriz de similitud en un DataFrame para un fácil acceso
similarity_matrix = pd.DataFrame(similarities, index=pivot_data.index, columns=pivot_data.index)

similarity_matrix.head()


id,10,20,30,40,50,60,70,80,130,220,...,901663,901667,901679,901735,901776,901805,2028055,2028056,2028103,2028850
id,,,,,,,,,,,,,,,,,,,,,
10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.707107,0.0,0.0,1.0,1.0,0.0,0.707107,1.0
20,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.707107,0.0,0.0,1.0,1.0,0.0,0.707107,1.0
30,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.707107,0.0,0.0,1.0,1.0,0.0,0.707107,1.0
40,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.707107,0.0,0.0,1.0,1.0,0.0,0.707107,1.0
50,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.707107,0.0,0.0,1.0,1.0,0.0,0.707107,1.0


Guardando el df de la matriz en un archivo parquet

In [43]:
# Guardar el DataFrame en un archivo parquet
import pickle

# Specify the file path where you want to save the DataFrame
file_path = 'dataRender//matriz.parquet'

# Save the DataFrame to a Parquet file
similarity_matrix.to_parquet(file_path, engine='pyarrow')

### Funcion de recomendacion de juegos

In [3]:
import pandas as pd

# Cargar el DataFrame con los datos de juegos
game_data = pd.read_parquet('dataRender//SistemaRecomendacion.parquet', engine='pyarrow')

def find_game_id_by_name(game_name):
    # Buscar el juego por nombre (insensible a mayúsculas y minúsculas)
    game_entry = game_data[game_data["app_name"].str.upper() == game_name.upper()]
    
    if not game_entry.empty:
        # Obtener el ID del juego encontrado
        game_id = game_entry["id"].values[0]
        return game_id
    else:
        print(f"No se encontró el juego con nombre '{game_name}' en la base de datos.")
        return None



def get_recommendations(game_name, n=5):
    try:
        game_id = find_game_id_by_name(game_name)
        if game_id == None:
           return f"El juego con ID {game_id} no se encuentra en la base de datos."
        # Cargar el DataFrame con la matriz de similitud
        similarity_matrix = pd.read_parquet('dataRender//matriz.parquet', engine='pyarrow')
        
        # Verificar si el juego de entrada (por ID) está en la base de datos
        if game_id in game_data["id"].values:
            # Obtener la fila de similitud para el juego de entrada
            input_game_similarity = similarity_matrix.loc[game_id]
            
            # Ordenar los juegos por similitud en orden descendente
            recommended_games = game_data.iloc[input_game_similarity.argsort()[::-1]]
            
            # Eliminar el juego de entrada de las recomendaciones
            recommended_games = recommended_games[recommended_games["id"] != game_id]
            
            # Eliminar recomendaciones duplicadas
            recommended_games = recommended_games.drop_duplicates(subset="id")
            
            # Seleccionar las primeras n recomendaciones
            top_n_recommendations = recommended_games.head(n)
            
            return top_n_recommendations
        else:
            #print(f"El juego con ID {game_id} no se encuentra en la base de datos.")
            return f"El juego con ID {game_id} no se encuentra en la base de datos."
    except Exception as e:
        print(f"Ocurrió un error: {str(e)}")
        return None
import pandas as pd


#*************************************************************************
# **************************************************************************    

# Ejemplo de uso para buscar el ID por nombre
input_game_name = "PLAYERUNKN1WN: FRIENDLY FIRE"  # Ingresa el nombre del juego
input_game_name = "Civilitation"
input_game_name ='LOST SUMMONER KITTY'
recommendations = get_recommendations(input_game_name)

display(recommendations)




,id,app_name,genres
2411,314380,LANTERN FORGE,RPG
9609,653890,RPG MAKER VX ACE - THE MUSIC BOX: JAPANESE HORROR,DESIGNAMPILLUSTRATION
11196,733080,PARTY HARD TYCOON OST,SIMULATION
11105,688620,TINY FORCE DELUXE,STRATEGY
4628,407620,INDIE GAME BATTLE,INDIE
